In [1]:
!pip install pycaret-ts-alpha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 468 kB 8.3 MB/s 
     |████████████████████████████████| 38.1 MB 2.0 MB/s 
     |████████████████████████████████| 3.3 MB 40.2 MB/s 
     |████████████████████████████████| 2.0 MB 41.5 MB/s 
     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 303 kB 42.7 MB/s 
     |████████████████████████████████| 11.2 MB 34.4 MB/s 
     |████████████████████████████████| 79.9 MB 78 kB/s 
     |████████████████████████████████| 120 kB 17.1 MB/s 
     |████████████████████████████████| 69 kB 7.3 MB/s 
     |████████████████████████████████| 1.4 MB 27.1 MB/s 
     |████████████████████████████████| 9.8 MB 30.1 MB/s 
     |████████████████████████████████| 6.5 MB 41.5 MB/s 
     |████████████████████████████████| 930 kB 40.3 MB/s 
     |████████████████████████████████| 34.5 MB 17 kB/s 
  Created wheel for pyod: filename=py

In [2]:
import pycaret

In [37]:
import pandas as pd
data = pd.read_csv('test1.csv')

In [38]:
import time
import numpy as np
import pandas as pd

from pycaret.datasets import get_data
from pycaret.time_series import TSForecastingExperiment

In [39]:
data['timestamp'] = pd.to_datetime(data['dt'])
data.head()

,dt,v_min,v_max,v_mean,timestamp
0,2022-05-25 18:00:00,4.077,4.080,4.0785,2022-05-25 18:00:00
1,2022-05-25 18:00:30,4.066,4.080,4.0730,2022-05-25 18:00:30
2,2022-05-25 18:01:00,4.066,4.069,4.0675,2022-05-25 18:01:00
3,2022-05-25 18:01:30,4.064,4.066,4.0650,2022-05-25 18:01:30
4,2022-05-25 18:02:00,4.065,4.071,4.0680,2022-05-25 18:02:00


In [40]:
# plot 
import plotly.express as px
fig = px.line(data, x="timestamp", y=['v_mean', ], title='test', template = 'plotly_dark')
fig.show()

In [41]:
data.set_index('timestamp', drop=True, inplace=True)

In [42]:
# resample timeseries to hourly 
data = data.resample('H').sum()
# creature features from date
data['day'] = [i.day for i in data.index]
data['day_name'] = [i.day_name() for i in data.index]
data['day_of_year'] = [i.dayofyear for i in data.index]
data['week_of_year'] = [i.weekofyear for i in data.index]
data['hour'] = [i.hour for i in data.index]
data['is_weekday'] = [i.isoweekday() for i in data.index]
data.head()

,v_min,v_max,v_mean,day,day_name,day_of_year,week_of_year,hour,is_weekday
timestamp,,,,,,,,,
2022-05-25 18:00:00,487.847,488.558,488.2025,25,Wednesday,145,21,18,3
2022-05-25 19:00:00,487.066,487.798,487.4320,25,Wednesday,145,21,19,3
2022-05-25 20:00:00,486.331,487.056,486.6935,25,Wednesday,145,21,20,3
2022-05-25 21:00:00,485.658,486.448,486.0530,25,Wednesday,145,21,21,3
2022-05-25 22:00:00,480.836,481.671,481.2535,25,Wednesday,145,21,22,3


In [43]:
data = data['v_mean']

In [44]:
data

timestamp
2022-05-25 18:00:00    488.2025
2022-05-25 19:00:00    487.4320
2022-05-25 20:00:00    486.6935
2022-05-25 21:00:00    486.0530
2022-05-25 22:00:00    481.2535
2022-05-25 23:00:00    484.7775
2022-05-26 00:00:00    484.2880
2022-05-26 01:00:00    483.9255
2022-05-26 02:00:00    479.2100
2022-05-26 03:00:00    482.6825
2022-05-26 04:00:00    482.4615
2022-05-26 05:00:00    481.7825
2022-05-26 06:00:00    481.5010
2022-05-26 07:00:00    481.2435
2022-05-26 08:00:00    481.0065
2022-05-26 09:00:00    480.2200
Freq: H, Name: v_mean, dtype: float64

In [45]:
#y = get_data('test', verbose=False)
# We want to forecast the next 12 months of data and we will use 3 fold cross-validation to test the models.
fh = 12 # or alternately fh = np.arange(1,13)
fold = 3
# Global Plot Settings
fig_kwargs={'renderer': 'notebook'}
eda = TSForecastingExperiment()
eda.setup(data=data,target='v_mean')

,Description,Value
0,session_id,3859
1,Target,v_mean
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Data shape,"(16, 1)"
5,Train data shape,"(15, 1)"
6,Test data shape,"(1, 1)"
7,Fold Generator,ExpandingWindowSplitter
8,Fold Number,3
9,Enforce Prediction Interval,False


In [46]:
from pycaret.time_series import *
setup(data=data,target='v_mean', session_id = 123)

,Description,Value
0,session_id,123
1,Target,v_mean
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Data shape,"(16, 1)"
5,Train data shape,"(15, 1)"
6,Test data shape,"(1, 1)"
7,Fold Generator,ExpandingWindowSplitter
8,Fold Number,3
9,Enforce Prediction Interval,False


In [47]:
plot_model(plot = 'ts')

In [48]:
best = compare_models()

,Model,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,TT (Sec)
naive,Naive Forecaster,0.2587,0.2587,0.0005,0.0005,0.1494,0.1051,1.1933
theta,Theta Forecaster,0.2969,0.2969,0.0006,0.0006,0.1713,0.1206,0.0567
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Det...,0.4217,0.4217,0.0009,0.0009,0.2471,0.1728,0.0633
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.419,0.419,0.0009,0.0009,0.2456,0.1717,0.0367
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.491,0.491,0.001,0.001,0.2855,0.2003,0.2100
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,0.5599,0.5599,0.0012,0.0012,0.3249,0.2282,0.0833
par_cds_dt,Passive Aggressive w/ Cond. Deseasonalize & De...,0.577,0.577,0.0012,0.0012,0.3312,0.2337,0.0400
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,0.6212,0.6212,0.0013,0.0013,0.3556,0.2511,0.2700
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.6386,0.6386,0.0013,0.0013,0.3694,0.2597,0.0400
auto_arima,Auto ARIMA,0.6901,0.6901,0.0014,0.0014,0.3818,0.2735,0.3567


In [49]:
prophet = create_model('rf_cds_dt')
print(prophet)

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE
0,2022-05-26 05:00,0.8545,0.8545,0.0018,0.0018,0.4605,0.3336
1,2022-05-26 06:00,0.4846,0.4846,0.0010,0.0010,0.2810,0.1975
2,2022-05-26 07:00,0.5245,0.5245,0.0011,0.0011,0.3255,0.2224
Mean,NaT,0.6212,0.6212,0.0013,0.0013,0.3556,0.2511
SD,NaT,0.1658,0.1658,0.0003,0.0003,0.0763,0.0592


BaseCdsDtForecaster(regressor=RandomForestRegressor(n_jobs=-1,
                                                    random_state=123),
                    window_length=1)


In [50]:
tuned_prophet = tune_model(prophet)
print(tuned_prophet)

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE
0,2022-05-26 05:00,0.4627,0.4627,0.0010,0.0010,0.2493,0.1806
1,2022-05-26 06:00,0.5244,0.5244,0.0011,0.0011,0.3041,0.2137
2,2022-05-26 07:00,0.4969,0.4969,0.0010,0.0010,0.3083,0.2107
Mean,NaT,0.4947,0.4947,0.0010,0.0010,0.2872,0.2017
SD,NaT,0.0252,0.0252,0.0001,0.0001,0.0269,0.0149


BaseCdsDtForecaster(deseasonal_model='multiplicative',
                    regressor=RandomForestRegressor(bootstrap=False,
                                                    max_depth=4,
                                                    max_features='log2',
                                                    min_impurity_decrease=0.029838948304784174,
                                                    n_estimators=106, n_jobs=-1,
                                                    random_state=123),
                    window_length=1)


In [51]:
plot_model(best, plot = 'forecast')

In [52]:
plot_model(best, plot = 'forecast', data_kwargs = {'fh' : 30})

In [53]:
# init setup
from pycaret.anomaly import *
s = setup(data, session_id = 123)#,target = 'v_mean',)

AttributeError: ignored

In [ ]:
data.dtypes

In [ ]:
data

In [ ]:
from pycaret.time_series import *
setup(data, target='day', fold = 3,)